In [7]:
from atlas.Atlas import Atlas,AtlasInitiator
from atlas.NgSegmentMaker import NgSegmentMaker
from atlas.Assembler import AtlasAssembler
from Controllers.SqlController import SqlController
import numpy as np
from registration.utilities import get_similarity_transformation_from_dicts
from registration.algorithm import brain_to_atlas_transform, umeyama
from atlas.NgSegmentMaker import AtlasNgMaker


In [6]:
atlas = 'atlasV8'
debug = False
maker = AtlasNgMaker(atlas, debug, threshold=0.9)
# maker.assembler.assemble_all_structure_volume()
# maker.create_atlas_neuroglancer()

No histology for Atlas
No histology for Atlas


ValueError: zero-size array to reduction operation minimum which has no identity

In [9]:
self = Atlas()
self.load_volumes()
self.gaussian_filter_volumes(sigma = 3)
self.threshold = 0.8
self.threshold_volumes()
self.volumes = self.thresholded_volumes
self.com = self.get_average_coms()
self.structures = list(self.COM.keys())
self.convert_unit_of_com_dictionary(self.COM,[1,1,1])
self.origins = self.get_origin_from_coms()

No histology for Atlas


/scratch/programming/pipeline/pipeline/registration/algorithm.py:35: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/share/pipeline/lib/python3.6/site-packages/numpy/lib/function_base.py:380: RuntimeWarning:

Mean of empty slice.

/usr/local/share/pipeline/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: zero-size array to reduction operation minimum which has no identity

In [13]:
self.volumes

{'10N_L': array([[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],
 
        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False

In [10]:
from scipy.ndimage.measurements import center_of_mass
assert(hasattr(self, 'COM'))
assert(hasattr(self, 'volumes'))
shared_structures = set(self.COM.keys()).intersection(self.volumes.keys())
volume_coms = np.array([center_of_mass(self.volumes[si]) for si in shared_structures]).astype(int)
average_com = np.array(list(self.COM.values()))
origins = average_com - volume_coms
origins = (origins - origins.min(0)).astype(int) + 10
values = [self.volumes[ki] for ki in shared_structures]
self.volumes = dict(zip(shared_structures,values))
dict(zip(self.COM.keys(), origins))

ValueError: zero-size array to reduction operation minimum which has no identity